In [4]:
import requests
import pandas as pd
import numpy as np
from itertools import chain
from datetime import date
import numpy as np
import os
from decimal import Decimal
import re

pd.set_option("display.max_columns", None)

In [61]:
#hoje = date.today()
lista_respostas = []
tamanho_retorno = 1
df_list = []
pagina = 1
while tamanho_retorno > 0:
    API_VARS = {
        "type":"2",
        "date_from":'2022-08-01',
        "date_to":'2024-08-31',
        "store_status":"all",
        "additional":"active",
        "good_rating":"1",
        "neutral_rating": "1",
        "bad_rating":"1",
        "rows_limit":"5000",
        "page":f"{pagina}"
    }


    url = f"https://api.solucx.com.br/public/transaction?type={API_VARS['type']}&date_from={API_VARS['date_from']}&date_to={API_VARS['date_to']}&store_status={API_VARS['store_status']}&additional={API_VARS['additional']}&good_rating={API_VARS['good_rating']}&neutral_rating={API_VARS['neutral_rating']}&bad_rating={API_VARS['bad_rating']}&rows_limit={API_VARS['rows_limit']}&page={API_VARS['page']}"

    payload={}
    headers = {
    'x-solucx-api-key': '6b40168f4a4b046ad708e8599d838021',
    'x-solucx-user-token': 'Qu0Ao483G33y9P2Tdp78'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    resposta = response.json()
    lista_respostas.append(resposta)
    tamanho_retorno = len(resposta)
    pagina+=1


lista_resposta_1d = list(chain.from_iterable( lista_respostas))

In [62]:
lista_valores = []
for v in lista_resposta_1d:
    valor = dict(v)
    positivos = []
    negativos = []
    try:
        positivos = valor['rating']['motives']['positive']
    except:
        pass
    try:
        negativos = valor['rating']['motives']['negative']
    except:
        pass
    #-----------------STORE--------------------------------------------------
    try:
        valor['store_id'] = valor['store']['id']
    except:
        pass
    try:
        valor['store_name'] = valor['store']['name']
    except:
        pass
    try:
        valor['store_external_id'] = valor['store']['external_id']
    except:
        pass
    #-----------------EMPLOYEE--------------------------------------------------
    try:
        valor['employee_id'] = valor['employee']['id']
    except:
        pass
    try:
        valor['employee_name'] = valor['employee']['name']
    except:
        pass
    try:
        valor['employee_external_id'] = valor['employee']['external_id']
    except:
        pass
    
    #-----------------EXTRAS--------------------------------------------------
    try:    
        valor['extras_bandeira'] = valor['extras']['bandeira']
    except:
        pass
    try:
        valor['extras_id_filial'] = valor['extras']['id_filial']
    except:
        pass
    try:
        valor['extras_cidade'] = valor['extras']['cidade']
    except:
        pass
    try:
        valor['extras_departamento'] = valor['extras']['departamento']
    except:
        pass
    try:
        valor['extras_setor1'] = valor['extras']['setor1']
    except:
        pass
    try:
        valor['extras_tipo_cliente'] = valor['extras']['tipo_cliente']
    except:
        pass
    try:
        valor['extras_funcionario'] = valor['extras']['funcionario']
    except:
        pass
    try:
        valor['extras_horario_transacao'] = valor['extras']['horario_transacao']
    except:
        pass    

    #---------------------------CUSTOMER--------------------------------------
    try:
        valor['customer_id'] = valor['customer']['id']
    except:
        pass
    try:
        valor['customer_name'] = valor['customer']['name']
    except:
        pass
    try:
        valor['customer_external_id'] = valor['customer']['external_id']
    except:
        pass
    try:
        valor['customer_gender'] = valor['customer']['gender']
    except:
        pass
    try:
        valor['customer_email'] = valor['customer']['email']
    except:
        pass
    try:
        valor['customer_valid_email'] = valor['customer']['valid_email']
    except:
        pass
    try:
        valor['customer_phone'] = valor['customer']['phone']
    except:
        pass
    try:
        valor['customer_cpf'] = valor['customer']['cpf']
    except:
        pass    

    #-------------------------RATING-----------------------------------------------
    try:
        valor['rating_timestamp'] = valor['rating']['timestamp'] 
    except:
        pass
    try:
        valor['rating_last_update'] = valor['rating']['last_update']  
    except:
        pass  
    try:
        valor['rating_value'] = valor['rating']['value']     
    except:
        pass
    try:
        valor['rating_source'] = valor['rating']['source']    
    except:
        pass
    try:
        valor['rating_comment'] = valor['rating']['comment']    
    except:
        pass 
    try:
        valor['rating_comment_reply'] = valor['rating']['comment_reply']
    except:
        pass
      
    if len(positivos)> 0:
        for p in positivos:
            valor['pergunta'] = p
            valor['resposta'] = 'gostou'
            valor_add = valor.copy()
            lista_valores.append(valor_add)

    if len(negativos)>0:
        for n in negativos: 
            valor['pergunta'] = n
            valor['resposta'] = 'não gostou'
            valor_add = valor.copy()
            lista_valores.append(valor_add)
            
    if len(positivos) == 0 and len(negativos) == 0:
        valor_add = valor.copy()
        lista_valores.append(valor_add)

In [63]:
df_final = pd.DataFrame(lista_valores)

In [64]:
df_final = df_final.drop(columns=['aval_complete', 'store', 'customer', 'employee', 'extras', 'sent_survey', 'sent_block', 'survey_enrichments', 'rating'])


In [65]:
mapa = {
 'id':'id',
 'transaction_id': 'transaction_id',
 'external_id': 'external_id',
 'aval_complete': 'aval_complete',
 'type': 'tipo',
 'journey': 'jornada',
 'survey_type': 'survey_type',
 'quarantine': 'bloqueio de quarentena',
 'timestamp': 'criado_em',
 'amount': 'valor',
 'reversed': 'estornado',
 'store_id': 'id_loja',
 'store_name': 'unidade',
 'store_external_id': 'ref_unidade',
 'employee_id': 'ref_colaborador',
 'employee_name': 'colaborador',
 'employee_external_id': 'employee_external_id',
 'extras_bandeira': 'bandeira',
 'extras_id_filial': 'id_filial',
 'extras_cidade': 'cidade',
 'extras_departamento': 'departamento',
 'extras_setor1': 'setor1',
 'extras_tipo_cliente': 'tipo_cliente',
 'extras_funcionario': 'funcionario',
 'extras_horario_transacao': 'horario_transacao',
 'customer_id': 'id_cliente',
 'customer_name': 'cliente',
 'customer_external_id': 'ref_cliente',
 'customer_gender': 'sexo',
 'customer_email': 'email',
 'customer_valid_email': 'email_valido',
 'customer_phone': 'telefone',
 'customer_cpf': 'cpf',
 'rating_timestamp': 'avaliado_em',
 'rating_last_update': 'avaliado_edit',
 'rating_value': 'nota',
 'rating_source': 'canal de resposta',
 'rating_comment': 'comentario',
 'rating_comment_reply': 'rating_comment_reply',
 'pergunta': 'pergunta',
 'resposta': 'resposta'
}

In [66]:
df = df_final.rename(columns=mapa)

In [67]:
df_raw = df.copy()

In [68]:
# Função para aplicar a lógica desejada
def separar_letra_numero(valor):
    if pd.notna(valor) and isinstance(valor, str) and any(c.isalpha() for c in valor):
        return pd.Series([x for x in re.split(r'([A-Za-z]+)?-?(\d+)?', valor) if x])
    else:
        return pd.Series([valor, valor])

In [69]:
# Aplica a função às linhas do DataFrame
df_raw[['Coluna1', 'Coluna2']] = df_raw['ref_cliente'].apply(separar_letra_numero)

In [70]:
# Criar duas novas colunas
df_raw[['Coluna1', 'Coluna2']] = df_raw['ref_cliente'].str.extract(r'([A-Za-z]+)?-?(\d+)?')

In [71]:
# Lista de colunas a serem excluídas
colunas_a_excluir = ['external_id', 'tipo', 'survey_type', 'bloqueio de quarentena', 'valor', 'estornado', 
                     'ref_colaborador', 'colaborador', 'employee_external_id', 'ref_cliente', 'sexo', 'email_valido', 
                     'cpf', 'rating_comment_reply', 'Coluna1']

In [72]:
# Excluir as colunas
df_raw = df_raw.drop(columns=colunas_a_excluir, errors='ignore')

In [73]:
# Renomear a coluna 'Coluna2' para 'ref_cliente'
df_raw = df_raw.rename(columns={'Coluna2': 'ref_cliente'})

In [74]:
# Deixando a coluna 'criado_em' no formato de data
df_raw['criado_em'] = pd.to_datetime(df_raw['criado_em'])

In [75]:
# Criando coluna para armazenar o ano e mês da 'criado_em'
df_raw['ano_mes'] =df_raw['criado_em'].dt.to_period('M')

In [76]:
# Criando coluna para armazenar o ano da 'criado_em'
df_raw['ano'] = df_raw['criado_em'].dt.year

In [77]:
df_raw.pergunta.unique()

array([nan, 'Preço justo (bom custo benefício)',
       'Variedade de produtos no açougue',
       'Atendimento operador de caixa',
       'Variedade de produtos no hortifruti',
       'Organização da loja (limpeza, organização de produtos nas prateleiras, acesso aos corredores, disponibilidade de carrinhos)',
       'Tempo de espera no caixa',
       'Preço no caixa igual ao da prateleira', 'Variedade de Produtos',
       'Qualidade de produtos no hortifruti',
       'Variedade de produtos na padaria',
       'Qualidade de produtos no açougue',
       'Limpeza e organização dos banheiros', 'Clube Amigão',
       'Estrutura da loja (facilidade de acesso, estacionamento, iluminação, som ambiente, climatização)',
       'Qualidade de produtos na padaria',
       'Facilidade no acompanhamento do pedido',
       'Cordialidade do entregador', 'Entrega dentro do prazo?',
       'Todos os itens foram entregues?', 'Ofertas',
       'Prazo proposto para entrega', 'Preço',
       'Cuidado na sep

In [78]:
df_raw['id_loja'] = df_raw['id_loja'].astype(int)

In [79]:
df_raw.jornada.unique()

array(['nps_amigao_whatsapp', 'nps_ecommerce_whatsapp', 'nps_amigao',
       'nps_ecommerce_am', 'gerenciamento_de_categorias',
       'nps_ecommerce_site', 'nps_stock_pf', 'nps_stock_pf_whatsapp',
       'nps_stock_pj', 'nps_stock_app', 'nps_cc', 'nps_ecommerce_cc',
       'nps_sf', 'nps_ecommerce_sf'], dtype=object)

In [80]:
df_varejo = df_raw.copy()

In [81]:
df_varejo

,id,transaction_id,jornada,criado_em,id_loja,unidade,ref_unidade,bandeira,id_filial,cidade,departamento,setor1,tipo_cliente,funcionario,horario_transacao,id_cliente,cliente,email,telefone,avaliado_em,avaliado_edit,nota,canal de resposta,comentario,pergunta,resposta,ref_cliente,ano_mes,ano
0,77483831,CSD-7710262024-08-nps_amigao_wha,nps_amigao_whatsapp,2024-08-31,10688,AMIGAO - GUAIAPO,AM_GUAIAPO,AM,38,MARINGA,FLV,"AV GUAIAPO N.1881,BOM JARDIM,MARINGA,PR",PF,N,18:27:00,3503315,DARCI CLEONICE SALOMAO,,44999541510,2024-09-02 11:46:52,2024-09-02 11:46:52,10,Whatsapp,False,NaN,NaN,771026,2024-08,2024
1,77483918,CSD-6312282024-08-nps_amigao_wha,nps_amigao_whatsapp,2024-08-31,10686,AMIGAO - CORNELIO II,AM_CORNELIOII,AM,30,CORNÉLIO PROCÓPIO,AÇOUGUE,"RUA BENJAMIN CONSTANT N.669,CENTRO,CORNÉLIO PR...",PF,N,19:58:00,3666969,ALESSANDRO HENRIQUE TOLEDO,halessandro005@gmail.com,43996126400,2024-09-02 11:00:25,2024-09-02 11:00:25,10,Whatsapp,False,Preço justo (bom custo benefício),gostou,631228,2024-08,2024
2,77483918,CSD-6312282024-08-nps_amigao_wha,nps_amigao_whatsapp,2024-08-31,10686,AMIGAO - CORNELIO II,AM_CORNELIOII,AM,30,CORNÉLIO PROCÓPIO,AÇOUGUE,"RUA BENJAMIN CONSTANT N.669,CENTRO,CORNÉLIO PR...",PF,N,19:58:00,3666969,ALESSANDRO HENRIQUE TOLEDO,halessandro005@gmail.com,43996126400,2024-09-02 11:00:25,2024-09-02 11:00:25,10,Whatsapp,False,Variedade de produtos no açougue,gostou,631228,2024-08,2024
3,77483918,CSD-6312282024-08-nps_amigao_wha,nps_amigao_whatsapp,2024-08-31,10686,AMIGAO - CORNELIO II,AM_CORNELIOII,AM,30,CORNÉLIO PROCÓPIO,AÇOUGUE,"RUA BENJAMIN CONSTANT N.669,CENTRO,CORNÉLIO PR...",PF,N,19:58:00,3666969,ALESSANDRO HENRIQUE TOLEDO,halessandro005@gmail.com,43996126400,2024-09-02 11:00:25,2024-09-02 11:00:25,10,Whatsapp,False,Atendimento operador de caixa,gostou,631228,2024-08,2024
4,77483918,CSD-6312282024-08-nps_amigao_wha,nps_amigao_whatsapp,2024-08-31,10686,AMIGAO - CORNELIO II,AM_CORNELIOII,AM,30,CORNÉLIO PROCÓPIO,AÇOUGUE,"RUA BENJAMIN CONSTANT N.669,CENTRO,CORNÉLIO PR...",PF,N,19:58:00,3666969,ALESSANDRO HENRIQUE TOLEDO,halessandro005@gmail.com,43996126400,2024-09-02 11:00:25,2024-09-02 11:00:25,10,Whatsapp,False,Variedade de produtos no hortifruti,gostou,631228,2024-08,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932944,18408983,039_2022-08-01_167560_010,nps_amigao,2022-08-01,10670,AMIGAO - OURINHOS,AM_OURINHOS,AM,39,OURINHOS,FLV,"RUA CARDOSO RIBEIRO N.970,CENTRO,OURINHOS,SP",NaN,NaN,NaN,322783,ALINE DE ALMEIDA DOMINGOS SILVA,almeida_alinedomingos@hotmail.com,14998736214,2022-08-04 17:38:33,2022-08-04 17:38:33,7,E-Mail,False,Preço,gostou,8612320517,2022-08,2022
932945,18416687,032_2022-08-01_165137_003,nps_cc,2022-08-01,10696,AMIGAO - NILDO RIBEIRO,AM_NILDORIBEIRO,CC,32,MARINGA,GERAL,"AV ARQUITETO NILDO RIBEIRO ROCHA N.343,JARDIM ...",NaN,NaN,NaN,244363,CARLOS MOURÃO,moharald@terra.com.br,44998851803,2022-08-03 21:14:05,2022-08-03 21:14:05,7,E-Mail,Não subir os preço no sabados e Domingos. Ai v...,Qualidade de produtos no AÇOUGUE,gostou,32525,2022-08,2022
932946,18416687,032_2022-08-01_165137_003,nps_cc,2022-08-01,10696,AMIGAO - NILDO RIBEIRO,AM_NILDORIBEIRO,CC,32,MARINGA,GERAL,"AV ARQUITETO NILDO RIBEIRO ROCHA N.343,JARDIM ...",NaN,NaN,NaN,244363,CARLOS MOURÃO,moharald@terra.com.br,44998851803,2022-08-03 21:14:05,2022-08-03 21:14:05,7,E-Mail,Não subir os preço no sabados e Domingos. Ai v...,Qualidade de produtos na PADARIA,gostou,32525,2022-08,2022
932947,18416687,032_2022-08-01_165137_003,nps_cc,2022-08-01,10696,AMIGAO - NILDO RIBEIRO,AM_NILDORIBEIRO,CC,32,MARINGA,GERAL,"AV ARQUITETO NILDO RIBEIRO ROCHA N.343,JARDIM ...",NaN,NaN,NaN,244363,CARLOS MOURÃO,moharald@terra.com.br,44998851803,2022-08-03 21:14:05,2022-08-03 21:14:05,7,E-Mail,Não subir os preço no sabados e Domingos. Ai v...,Estrutura da Loja,gostou,32525,2022-08,2022


In [82]:
# Filtro na coluna de pergunta apenas com "Preço" e só varejo
df_varejo = df_varejo[
    (df_varejo['jornada'] =='nps_amigao')|
    (df_varejo['jornada'] =='nps_cc')|
    (df_varejo['jornada'] =='nps_sf')|
    (df_varejo['jornada'] =='nps_amigao_whatsapp')|
    (df_varejo['jornada'] =='gerenciamento_de_categorias')
    ]

In [83]:
df_varejo['Regionais'] = np.select(
    [
        df_varejo['id_loja'].isin([10682, 10683, 10684, 10691, 10692, 10696, 10703, 10706, 10709]),  # REGIONAL 1
        df_varejo['id_loja'].isin([10677, 10688, 10690, 10693, 10694, 10695, 10698, 10699, 10708, 10711]),  # REGIONAL 2
        df_varejo['id_loja'].isin([10687, 10697, 10700, 10702, 10710, 10712, 10713, 10715, 21493, 21473]),  # REGIONAL 3
        df_varejo['id_loja'].isin([10678, 10679, 10680, 10681, 10685, 10686, 10689, 10704, 10707, 10714, 10724, 10660]),  # REGIONAL 4
        df_varejo['id_loja'].isin([10660, 10667, 10671, 10674, 10675, 10716, 21475, 21498, 21474]),  # REGIONAL 5
        df_varejo['id_loja'].isin([10662, 10663, 10672, 10676, 10717, 10718, 10719]),  # REGIONAL 6
        df_varejo['id_loja'].isin([10661, 10664, 10665, 10666, 10668, 10669, 10670, 10673, 21476]),  # REGIONAL 7
        df_varejo['id_loja'].isin([21466, 21467, 21468, 21469, 21470, 21471, 21472, 21489])  # REGIONAL 8
    ],
    [
        "REGIONAL 1 - MARINGÁ 1",
        "REGIONAL 2 - MARINGÁ 2",
        "REGIONAL 3 - UMUARAMA",
        "REGIONAL 4 - LONDRINA",
        "REGIONAL 5 - ARAÇATUBA",
        "REGIONAL 6 - SJRP",
        "REGIONAL 7 - LINS",
        "REGIONAL 8 - STOCK"
    ],
    default="SEM GO"
)

C:\Users\Bruno.Freitas\AppData\Local\Temp\ipykernel_15528\3680715788.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_varejo['Regionais'] = np.select(


In [84]:
df_varejo.pergunta.unique()

array([nan, 'Preço justo (bom custo benefício)',
       'Variedade de produtos no açougue',
       'Atendimento operador de caixa',
       'Variedade de produtos no hortifruti',
       'Organização da loja (limpeza, organização de produtos nas prateleiras, acesso aos corredores, disponibilidade de carrinhos)',
       'Tempo de espera no caixa',
       'Preço no caixa igual ao da prateleira', 'Variedade de Produtos',
       'Qualidade de produtos no hortifruti',
       'Variedade de produtos na padaria',
       'Qualidade de produtos no açougue',
       'Limpeza e organização dos banheiros', 'Clube Amigão',
       'Estrutura da loja (facilidade de acesso, estacionamento, iluminação, som ambiente, climatização)',
       'Qualidade de produtos na padaria',
       'Encontrou tudo que procurava?',
       'Cuidado e higiene com os produtos', 'Ofertas',
       'Sinalização do preço na prateleira',
       'Qualidade de produtos na PADARIA',
       'Qualidade de produtos no AÇOUGUE',
       'Qu

In [ ]:
# Filtro na coluna de pergunta com: 

# Sinalização do preço na prateleira, 
# Sinalização do preço nas prateleiras, 
# Sinalização das ofertas, 
# Preço no caixa igual o da prateleira?
# Preço no caixa igual ao da prateleira

In [87]:
# Filtro na coluna de pergunta apenas com "Ofertas" e só varejo
df_oferta = df_varejo[
    ((df_varejo['pergunta'] =='Ofertas') |
    (df_varejo['pergunta'] =='Promoção'))
    ]

In [88]:
# 1º) QUantidade distinta de 'id' que deram "gostou"
likes_oferta_rede = df_oferta[df_oferta['resposta'] == 'gostou'].groupby('ano')['id'].nunique()
likes_oferta_rede

ano
2022     1743
2023     2975
2024    12813
Name: id, dtype: int64

In [89]:
# 2º) QUantidade distinta de 'id' que deram "não gostou"
dislikes_oferta_rede = df_oferta[df_oferta['resposta'] == 'não gostou'].groupby('ano')['id'].nunique()
dislikes_oferta_rede

ano
2022     362
2023     634
2024    3191
Name: id, dtype: int64

In [90]:
# 3º) QUantidade distinta de 'id' que deram "gostou" por loja
likes_oferta_loja = df_oferta[df_oferta['resposta'] == 'gostou'].groupby(['ano','unidade'])['id'].nunique()
likes_oferta_loja

ano   unidade             
2022  AMIGAO - ALVORADA        13
      AMIGAO - APUCARANA       30
      AMIGAO - ARACATUBA       16
      AMIGAO - ARAPONGAS       20
      AMIGAO - ASSIS           52
                             ... 
2024  AMIGAO - TUIUTI         130
      AMIGAO - TUPA           310
      AMIGAO - UMUARAMA I     224
      AMIGAO - UMUARAMA II    270
      AMIGAO - ZONA NORTE     183
Name: id, Length: 194, dtype: int64

In [91]:
# 4º) QUantidade distinta de 'id' que deram "não gostou" por loja
dislikes_oferta_loja = df_oferta[df_oferta['resposta'] == 'não gostou'].groupby(['ano','unidade'])['id'].nunique()
dislikes_oferta_loja

ano   unidade             
2022  AMIGAO - ALVORADA        6
      AMIGAO - APUCARANA       6
      AMIGAO - ARACATUBA       6
      AMIGAO - ARAPONGAS       4
      AMIGAO - ASSIS          10
                              ..
2024  AMIGAO - TUIUTI         53
      AMIGAO - TUPA           63
      AMIGAO - UMUARAMA I     49
      AMIGAO - UMUARAMA II    50
      AMIGAO - ZONA NORTE     46
Name: id, Length: 193, dtype: int64

In [92]:
# 3º) Percentual de likes REDE (oferta)
percent_likes_oferta_rede = (likes_oferta_rede/(likes_oferta_rede+dislikes_oferta_rede))
percent_likes_oferta_rede

ano
2022    0.828029
2023    0.824328
2024    0.800612
Name: id, dtype: float64

In [93]:
# 3º) Percentual de likes Loja (oferta)
percent_likes_oferta_loja = (likes_oferta_loja/(likes_oferta_loja+dislikes_oferta_loja))
percent_likes_oferta_loja

ano   unidade             
2022  AMIGAO - ALVORADA       0.684211
      AMIGAO - APUCARANA      0.833333
      AMIGAO - ARACATUBA      0.727273
      AMIGAO - ARAPONGAS      0.833333
      AMIGAO - ASSIS          0.838710
                                ...   
2024  AMIGAO - TUIUTI         0.710383
      AMIGAO - TUPA           0.831099
      AMIGAO - UMUARAMA I     0.820513
      AMIGAO - UMUARAMA II    0.843750
      AMIGAO - ZONA NORTE     0.799127
Name: id, Length: 194, dtype: float64

In [94]:
percent_likes_oferta_rede.to_excel(r"C:\Users\Bruno.Freitas\Downloads\analise_24m_NPS\oferta_rede_ano.xlsx")

In [95]:
percent_likes_oferta_loja.to_excel(r"C:\Users\Bruno.Freitas\Downloads\analise_24m_NPS\oferta_loja_ano.xlsx")

In [ ]:
# # Filtro na coluna de pergunta apenas com "Preço" e só varejo
# df_sinalizacao = df_varejo[
#     ((df_varejo['pergunta'] =='Sinalização do preço na prateleira') |
#     (df_varejo['pergunta'] =='Sinalização do preço nas prateleiras') |
#     (df_varejo['pergunta'] =='Sinalização das ofertas'))
#     ]

In [131]:
# Filtro na coluna de pergunta apenas com "Preço" e só varejo
df_sinalizacao1 = df_varejo[
    (df_varejo['pergunta'] =='Sinalização das ofertas')
    ]

In [132]:
# Filtro na coluna de pergunta apenas com "Preço" e só varejo
df_sinalizacao = df_varejo[
    ((df_varejo['pergunta'] =='Sinalização do preço na prateleira') |
    (df_varejo['pergunta'] =='Sinalização do preço nas prateleiras'))
    ]

In [133]:
# 1º) QUantidade distinta de 'id' que deram "gostou"
likes_sinal_rede = df_sinalizacao[df_sinalizacao['resposta'] == 'gostou'].groupby('ano_mes')['id'].nunique()
likes_sinal_rede

ano_mes
2023-05       6
2023-06     163
2023-07     188
2023-08     121
2023-09      82
2023-10     113
2023-11     224
2023-12     378
2024-01    1505
2024-02    1387
2024-03    1333
2024-04    1302
2024-05    1681
2024-06    1285
2024-07    1547
2024-08    1438
Freq: M, Name: id, dtype: int64

In [134]:
# 1º) QUantidade distinta de 'id' que deram "gostou"
likes_sinal1_rede = df_sinalizacao1[df_sinalizacao1['resposta'] == 'gostou'].groupby('ano_mes')['id'].nunique()
likes_sinal1_rede

ano_mes
2022-08    274
2022-09    360
2022-10    329
2022-11    383
2022-12    423
2023-01    431
2023-02    447
2023-03    359
2023-04    189
2023-05    192
Freq: M, Name: id, dtype: int64

In [135]:
# 2º) QUantidade distinta de 'id' que deram "não gostou"
dislikes_sinal_rede = df_sinalizacao[df_sinalizacao['resposta'] == 'não gostou'].groupby('ano_mes')['id'].nunique()
dislikes_sinal_rede

ano_mes
2023-06     48
2023-07     61
2023-08     28
2023-09     18
2023-10     37
2023-11     81
2023-12    123
2024-01    582
2024-02    554
2024-03    503
2024-04    497
2024-05    704
2024-06    445
2024-07    491
2024-08    509
Freq: M, Name: id, dtype: int64

In [136]:
# 2º) QUantidade distinta de 'id' que deram "não gostou"
dislikes_sinal1_rede = df_sinalizacao1[df_sinalizacao1['resposta'] == 'não gostou'].groupby('ano_mes')['id'].nunique()
dislikes_sinal1_rede

ano_mes
2022-08    59
2022-09    61
2022-10    62
2022-11    58
2022-12    73
2023-01    72
2023-02    83
2023-03    59
2023-04    34
2023-05    35
Freq: M, Name: id, dtype: int64

In [137]:
# 3º) QUantidade distinta de 'id' que deram "gostou" por loja
likes_sinal_loja = df_sinalizacao[df_sinalizacao['resposta'] == 'gostou'].groupby(['ano_mes','Regionais'])['id'].nunique()
likes_sinal_loja

ano_mes  Regionais             
2023-05  REGIONAL 3 - UMUARAMA       4
         REGIONAL 7 - LINS           2
2023-06  REGIONAL 1 - MARINGÁ 1     26
         REGIONAL 2 - MARINGÁ 2     30
         REGIONAL 3 - UMUARAMA      16
                                  ... 
2024-08  REGIONAL 3 - UMUARAMA     287
         REGIONAL 4 - LONDRINA     334
         REGIONAL 5 - ARAÇATUBA    207
         REGIONAL 6 - SJRP         141
         REGIONAL 7 - LINS         192
Name: id, Length: 110, dtype: int64

In [138]:
# 3º) QUantidade distinta de 'id' que deram "gostou" por loja
likes_sinal1_loja = df_sinalizacao1[df_sinalizacao1['resposta'] == 'gostou'].groupby(['ano_mes','Regionais'])['id'].nunique()
likes_sinal1_loja

ano_mes  Regionais             
2022-08  REGIONAL 1 - MARINGÁ 1    26
         REGIONAL 2 - MARINGÁ 2    21
         REGIONAL 3 - UMUARAMA     88
         REGIONAL 4 - LONDRINA     36
         REGIONAL 5 - ARAÇATUBA    47
                                   ..
2023-05  REGIONAL 3 - UMUARAMA     33
         REGIONAL 4 - LONDRINA     40
         REGIONAL 5 - ARAÇATUBA    15
         REGIONAL 6 - SJRP         27
         REGIONAL 7 - LINS         26
Name: id, Length: 70, dtype: int64

In [139]:
# 4º) QUantidade distinta de 'id' que deram "não gostou" por loja
dislikes_sinal_loja = df_sinalizacao[df_sinalizacao['resposta'] == 'não gostou'].groupby(['ano_mes','Regionais'])['id'].nunique()
dislikes_sinal_loja

ano_mes  Regionais             
2023-06  REGIONAL 1 - MARINGÁ 1     9
         REGIONAL 2 - MARINGÁ 2    16
         REGIONAL 3 - UMUARAMA      3
         REGIONAL 4 - LONDRINA      5
         REGIONAL 5 - ARAÇATUBA     4
                                   ..
2024-08  REGIONAL 3 - UMUARAMA     70
         REGIONAL 4 - LONDRINA     95
         REGIONAL 5 - ARAÇATUBA    65
         REGIONAL 6 - SJRP         51
         REGIONAL 7 - LINS         85
Name: id, Length: 107, dtype: int64

In [140]:
# 4º) QUantidade distinta de 'id' que deram "não gostou" por loja
dislikes_sinal1_loja = df_sinalizacao1[df_sinalizacao1['resposta'] == 'não gostou'].groupby(['ano_mes','Regionais'])['id'].nunique()
dislikes_sinal1_loja

ano_mes  Regionais             
2022-08  REGIONAL 1 - MARINGÁ 1     8
         REGIONAL 2 - MARINGÁ 2     9
         REGIONAL 3 - UMUARAMA     15
         REGIONAL 4 - LONDRINA      8
         REGIONAL 5 - ARAÇATUBA     6
                                   ..
2023-05  REGIONAL 2 - MARINGÁ 2     9
         REGIONAL 3 - UMUARAMA      5
         REGIONAL 4 - LONDRINA     10
         REGIONAL 6 - SJRP          3
         REGIONAL 7 - LINS          3
Name: id, Length: 69, dtype: int64

In [151]:
# 3º) Percentual de likes REDE (Sinalização)
percent_likes_sinal_rede = likes_sinal_rede/(likes_sinal_rede+dislikes_sinal_rede)
percent_likes_sinal_rede

ano_mes
2023-05         NaN
2023-06    0.772512
2023-07    0.755020
2023-08    0.812081
2023-09    0.820000
2023-10    0.753333
2023-11    0.734426
2023-12    0.754491
2024-01    0.721131
2024-02    0.714580
2024-03    0.726035
2024-04    0.723735
2024-05    0.704822
2024-06    0.742775
2024-07    0.759078
2024-08    0.738572
Freq: M, Name: id, dtype: float64

In [153]:
# 3º) Percentual de likes REDE (Sinalização)
percent_likes_sinal1_rede = likes_sinal1_rede/(likes_sinal1_rede+dislikes_sinal1_rede)
percent_likes_sinal1_rede

ano_mes
2022-08    0.822823
2022-09    0.855107
2022-10    0.841432
2022-11    0.868481
2022-12    0.852823
2023-01    0.856859
2023-02    0.843396
2023-03    0.858852
2023-04    0.847534
2023-05    0.845815
Freq: M, Name: id, dtype: float64

In [154]:
# 3º) Percentual de likes Loja (Sinalização)
percent_likes_sinal_loja = (likes_sinal_loja/(likes_sinal_loja+dislikes_sinal_loja))
percent_likes_sinal_loja

ano_mes  Regionais             
2023-05  REGIONAL 3 - UMUARAMA          NaN
         REGIONAL 7 - LINS              NaN
2023-06  REGIONAL 1 - MARINGÁ 1    0.742857
         REGIONAL 2 - MARINGÁ 2    0.652174
         REGIONAL 3 - UMUARAMA     0.842105
                                     ...   
2024-08  REGIONAL 3 - UMUARAMA     0.803922
         REGIONAL 4 - LONDRINA     0.778555
         REGIONAL 5 - ARAÇATUBA    0.761029
         REGIONAL 6 - SJRP         0.734375
         REGIONAL 7 - LINS         0.693141
Name: id, Length: 110, dtype: float64

In [155]:
# 3º) Percentual de likes Loja (Sinalização)
percent_likes_sinal1_loja = (likes_sinal1_loja/(likes_sinal1_loja+dislikes_sinal1_loja))
percent_likes_sinal1_loja

ano_mes  Regionais             
2022-08  REGIONAL 1 - MARINGÁ 1    0.764706
         REGIONAL 2 - MARINGÁ 2    0.700000
         REGIONAL 3 - UMUARAMA     0.854369
         REGIONAL 4 - LONDRINA     0.818182
         REGIONAL 5 - ARAÇATUBA    0.886792
                                     ...   
2023-05  REGIONAL 3 - UMUARAMA     0.868421
         REGIONAL 4 - LONDRINA     0.800000
         REGIONAL 5 - ARAÇATUBA         NaN
         REGIONAL 6 - SJRP         0.900000
         REGIONAL 7 - LINS         0.896552
Name: id, Length: 70, dtype: float64

In [156]:
percent_likes_sinal_rede.to_excel(r"C:\Users\Bruno.Freitas\Downloads\analise_24m_NPS\SINAL_REDE_mes.xlsx")

In [157]:
percent_likes_sinal_loja.to_excel(r"C:\Users\Bruno.Freitas\Downloads\analise_24m_NPS\SINAL_LOJA_mes.xlsx")

In [ ]:
# FIM

In [43]:
# Filtro na coluna de pergunta apenas com "Preço" e só varejo
df_divergencia = df_varejo[
    ((df_varejo['pergunta'] =='Preço no caixa igual o da prateleira?') |
    (df_varejo['pergunta'] =='Preço no caixa igual ao da prateleira'))
    ]

In [45]:
df_divergencia.head(2)

,id,transaction_id,jornada,criado_em,id_loja,unidade,ref_unidade,bandeira,id_filial,cidade,departamento,setor1,tipo_cliente,funcionario,horario_transacao,id_cliente,cliente,email,telefone,avaliado_em,avaliado_edit,nota,canal de resposta,comentario,pergunta,resposta,ref_cliente,ano_mes,ano,Regionais
13,77483919,CSD-8012024512024-08-nps_amigao_,nps_amigao_whatsapp,2024-08-31,21493,AMIGAO - NAVIRAI,AM_NAVIRAI,AM,84,NAVIRAÍ,GERAL,"AV DOURADOS,CENTRO,NAVIRAÍ,MS",PF,N,11:14:00,4700419,CLEODIANE ALBUQUERQUE REZENDE,cleorezende1987@hotmail.com,67996653560,2024-09-02 11:27:08,2024-09-02 11:27:08,10,Whatsapp,False,Preço no caixa igual ao da prateleira,gostou,801202451,2024-08,2024,REGIONAL 3 - UMUARAMA
35,77484289,CSD-89987444302024-08-nps_amigao,nps_amigao_whatsapp,2024-08-31,10694,AMIGAO - MORANGUEIRA,AM_MORANGUEIRA,AM,9,MARINGA,GERAL,"AVENIDA MORANGUEIRA N.1505,VILA MORANGUEIRA,MA...",PF,N,11:22:00,3832403,ANGELO TAKEO TURUSHIMA,turushima@hotmail.com,44991387596,2024-09-02 12:05:06,2024-09-02 12:05:06,10,Whatsapp,False,Preço no caixa igual ao da prateleira,gostou,8998744430,2024-08,2024,REGIONAL 2 - MARINGÁ 2


In [51]:
# 1º) QUantidade distinta de 'id' que deram "gostou"
likes_diver_rede = df_divergencia[df_divergencia['resposta'] == 'gostou'].groupby('ano_mes')['id'].nunique()
likes_diver_rede

ano_mes
2023-05       6
2023-06     174
2023-07     204
2023-08     125
2023-09      81
2023-10     117
2023-11     237
2023-12     407
2024-01    1676
2024-02    1586
2024-03    1515
2024-04    1453
2024-05    1877
2024-06    1330
2024-07    1585
2024-08    1533
Freq: M, Name: id, dtype: int64

In [52]:
# 2º) QUantidade distinta de 'id' que deram "não gostou"
dislikes_diver_rede = df_divergencia[df_divergencia['resposta'] == 'não gostou'].groupby('ano_mes')['id'].nunique()
dislikes_diver_rede

ano_mes
2023-06     36
2023-07     44
2023-08     24
2023-09     20
2023-10     35
2023-11     68
2023-12    107
2024-01    389
2024-02    419
2024-03    381
2024-04    395
2024-05    498
2024-06    328
2024-07    387
2024-08    361
Freq: M, Name: id, dtype: int64

In [46]:
# 3º) QUantidade distinta de 'id' que deram "gostou" por loja
likes_diver_loja = df_divergencia[df_divergencia['resposta'] == 'gostou'].groupby(['ano_mes','Regionais'])['id'].nunique()
likes_diver_loja

ano_mes  Regionais             
2023-05  REGIONAL 3 - UMUARAMA       4
         REGIONAL 7 - LINS           2
2023-06  REGIONAL 1 - MARINGÁ 1     26
         REGIONAL 2 - MARINGÁ 2     33
         REGIONAL 3 - UMUARAMA      18
                                  ... 
2024-08  REGIONAL 3 - UMUARAMA     292
         REGIONAL 4 - LONDRINA     300
         REGIONAL 5 - ARAÇATUBA    249
         REGIONAL 6 - SJRP         160
         REGIONAL 7 - LINS         215
Name: id, Length: 110, dtype: int64

In [47]:
# 4º) QUantidade distinta de 'id' que deram "gostou" por loja
dislikes_diver_loja = df_divergencia[df_divergencia['resposta'] == 'não gostou'].groupby(['ano_mes','Regionais'])['id'].nunique()
dislikes_diver_loja

ano_mes  Regionais             
2023-06  REGIONAL 1 - MARINGÁ 1     9
         REGIONAL 2 - MARINGÁ 2    12
         REGIONAL 3 - UMUARAMA      1
         REGIONAL 4 - LONDRINA      4
         REGIONAL 5 - ARAÇATUBA     3
                                   ..
2024-08  REGIONAL 3 - UMUARAMA     47
         REGIONAL 4 - LONDRINA     70
         REGIONAL 5 - ARAÇATUBA    38
         REGIONAL 6 - SJRP         44
         REGIONAL 7 - LINS         49
Name: id, Length: 105, dtype: int64

In [54]:
# 5º) Percentual de likes REDE (Sinalização)
percent_likes_diver_rede = (likes_diver_rede/(likes_diver_rede+dislikes_diver_rede))
percent_likes_diver_rede

ano_mes
2023-05         NaN
2023-06    0.828571
2023-07    0.822581
2023-08    0.838926
2023-09    0.801980
2023-10    0.769737
2023-11    0.777049
2023-12    0.791829
2024-01    0.811622
2024-02    0.791022
2024-03    0.799051
2024-04    0.786255
2024-05    0.790316
2024-06    0.802171
2024-07    0.803753
2024-08    0.809398
Freq: M, Name: id, dtype: float64

In [55]:
# 3º) Percentual de likes Loja (Sinalização)
percent_likes_diver_loja = (likes_diver_loja/(likes_diver_loja+dislikes_diver_loja))
percent_likes_diver_loja

ano_mes  Regionais             
2023-05  REGIONAL 3 - UMUARAMA          NaN
         REGIONAL 7 - LINS              NaN
2023-06  REGIONAL 1 - MARINGÁ 1    0.742857
         REGIONAL 2 - MARINGÁ 2    0.733333
         REGIONAL 3 - UMUARAMA     0.947368
                                     ...   
2024-08  REGIONAL 3 - UMUARAMA     0.861357
         REGIONAL 4 - LONDRINA     0.810811
         REGIONAL 5 - ARAÇATUBA    0.867596
         REGIONAL 6 - SJRP         0.784314
         REGIONAL 7 - LINS         0.814394
Name: id, Length: 110, dtype: float64

In [56]:
percent_likes_diver_rede.to_excel(r"C:\Users\Bruno.Freitas\Downloads\analise_24m_NPS\divergencia_rede_mes.xlsx")

In [57]:
percent_likes_diver_loja.to_excel(r"C:\Users\Bruno.Freitas\Downloads\analise_24m_NPS\divergencia_regional_mes.xlsx")

In [105]:
# Filtro na coluna de pergunta apenas com "Sinalização do preço na prateleira" e "Sinalização do preço nas prateleiras" e só varejo
df_sinalizacao = df_raw[(df_raw['pergunta'].isin(['Sinalização do preço na prateleira', 'Sinalização do preço nas prateleiras','Sinalização das ofertas'])) &
                        ((df_raw['jornada'] == 'nps_amigao') |
                         (df_raw['jornada'] == 'nps_cc') |
                         (df_raw['jornada'] == 'nps_sf'))]

In [ ]:
# 1º) QUantidade distinta de 'id' que deram "gostou"
likes_sinal_rede = df_sinalizacao[df_sinalizacao['resposta'] == 'gostou'].groupby('ano_mes')['id'].nunique()
likes_sinal_rede

In [ ]:
# 1º) QUantidade distinta de 'id' que deram "gostou"
likes_sinal_reg = df_sinalizacao[df_sinalizacao['resposta'] == 'gostou'].groupby(['ano_mes','Regionais'])['id'].nunique()
likes_sinal_reg

In [ ]:
# 2º) QUantidade distinta de 'id' que deram "não gostou"
dislikes_sinal_rede = df_sinalizacao[df_sinalizacao['resposta'] == 'não gostou'].groupby('ano_mes')['id'].nunique()
dislikes_sinal_rede

In [ ]:
# 2º) QUantidade distinta de 'id' que deram "não gostou"
dislikes_sinal_reg = df_sinalizacao[df_sinalizacao['resposta'] == 'não gostou'].groupby(['ano_mes','Regionais'])['id'].nunique()
dislikes_sinal_reg

In [ ]:
# 3º) Percentual de likes (Preço)
percent_likes_sinal_rede = round((likes_sinal_rede/(likes_sinal_rede+dislikes_sinal_rede))*100,0)
percent_likes_sinal_rede

In [ ]:
# 3º) Percentual de likes (Preço)
percent_likes_sinal_reg = round((likes_sinal_reg/(likes_sinal_reg+dislikes_sinal_reg))*100,0)
percent_likes_sinal_reg

In [112]:
percent_likes_sinal_reg.to_excel(r"C:\Users\Bruno.Freitas\Downloads\sinal_antes_reg.xlsx")